### Quick Description

This notebook calculates the toxicity score from each sentence in a subtitles' file. Finally, it persists the data in a DataFrame like output.

In [1]:
%%capture
!pip install --upgrade google-api-python-client

In [2]:
import yaml
import glob
import os
import pandas as pd
import time
import re
import logging
import sys

from tqdm import tqdm
from googleapiclient import discovery

sys.path.append("../utils")
from toxicity_api_communication import get_toxicity_score

In [4]:
filepaths = yaml.load(open("../config/filepaths.yaml"))
credentials = yaml.load(open("../config/credentials.yaml"))["perspective-api"]

/home/guilherme-resende/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.
/home/guilherme-resende/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [5]:
def calculate_toxicity_scores(credentials, input_path, output_path):
    services = [
        discovery.build(
            "commentanalyzer",
            "v1alpha1",
            developerKey=value,
            discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
            static_discovery=False,
        ) for _, value in credentials.items()
    ]

    n_services = len(services)
    filenames = [file.split("/")[-1] for file in glob.glob(os.path.join(input_path, "*"))]

    for filename in tqdm(filenames, total=len(filenames)):
        df_sentences = pd.read_csv(os.path.join(input_path, filename))
        for i, row in df_sentences.iterrows():
            df_sentences.loc[i, "score"] = get_toxicity_score(
                services[i % n_services],
                row.text,
                "TOXICITY"
            )

            if (i % n_services) == 0:
                time.sleep(1)

        df_sentences.to_csv(os.path.join(output_path, filename), index=False)

In [6]:
calculate_toxicity_scores(
    credentials,
    filepaths["model_based_segmentated_data"],
    filepaths["scored_data"]
)

  3%|█▌                                                            | 14/552 [11:00<7:06:03, 47.52s/it]ERROR:root:The following error occured: 
({'vary': 'Origin, X-Origin, Referer', 'content-type': 'application/json; charset=UTF-8', 'date': 'Fri, 28 Jan 2022 10:57:57 GMT', 'server': 'ESF', 'cache-control': 'private', 'x-xss-protection': '0', 'x-frame-options': 'SAMEORIGIN', 'x-content-type-options': 'nosniff', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"', 'transfer-encoding': 'chunked', 'status': '400', 'content-length': '492', '-content-encoding': 'gzip'}, b'{\n  "error": {\n    "code": 400,\n    "message": "Attribute TOXICITY does not support request languages: ja-Latn",\n    "status": "INVALID_ARGUMENT",\n    "details": [\n      {\n        "@type": "type.googleapis.com/google.commentanalyzer.v1alpha1.Error",\n        "errorType": "LANGUAGE_NOT_SUPPORTED_

 26%|███████████████▎                                           | 143/552 [1:54:43<3:04:19, 27.04s/it]ERROR:root:The following error occured: 
({'vary': 'Origin, X-Origin, Referer', 'content-type': 'application/json; charset=UTF-8', 'date': 'Fri, 28 Jan 2022 12:40:11 GMT', 'server': 'ESF', 'cache-control': 'private', 'x-xss-protection': '0', 'x-frame-options': 'SAMEORIGIN', 'x-content-type-options': 'nosniff', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"', 'transfer-encoding': 'chunked', 'status': '400', 'content-length': '482', '-content-encoding': 'gzip'}, b'{\n  "error": {\n    "code": 400,\n    "message": "Attribute TOXICITY does not support request languages: tr",\n    "status": "INVALID_ARGUMENT",\n    "details": [\n      {\n        "@type": "type.googleapis.com/google.commentanalyzer.v1alpha1.Error",\n        "errorType": "LANGUAGE_NOT_SUPPORTED_BY_AT

 54%|███████████████████████████████▋                           | 296/552 [3:42:26<2:44:09, 38.47s/it]ERROR:root:The following error occured: 
({'vary': 'Origin, X-Origin, Referer', 'content-type': 'application/json; charset=UTF-8', 'date': 'Fri, 28 Jan 2022 14:28:10 GMT', 'server': 'ESF', 'cache-control': 'private', 'x-xss-protection': '0', 'x-frame-options': 'SAMEORIGIN', 'x-content-type-options': 'nosniff', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"', 'transfer-encoding': 'chunked', 'status': '400', 'content-length': '482', '-content-encoding': 'gzip'}, b'{\n  "error": {\n    "code": 400,\n    "message": "Attribute TOXICITY does not support request languages: bs",\n    "status": "INVALID_ARGUMENT",\n    "details": [\n      {\n        "@type": "type.googleapis.com/google.commentanalyzer.v1alpha1.Error",\n        "errorType": "LANGUAGE_NOT_SUPPORTED_BY_AT

 67%|███████████████████████████████████████▍                   | 369/552 [4:34:17<3:01:09, 59.39s/it]ERROR:root:The following error occured: 
({'vary': 'Origin, X-Origin, Referer', 'content-type': 'application/json; charset=UTF-8', 'date': 'Fri, 28 Jan 2022 15:20:32 GMT', 'server': 'ESF', 'cache-control': 'private', 'x-xss-protection': '0', 'x-frame-options': 'SAMEORIGIN', 'x-content-type-options': 'nosniff', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"', 'transfer-encoding': 'chunked', 'status': '400', 'content-length': '492', '-content-encoding': 'gzip'}, b'{\n  "error": {\n    "code": 400,\n    "message": "Attribute TOXICITY does not support request languages: ru-Latn",\n    "status": "INVALID_ARGUMENT",\n    "details": [\n      {\n        "@type": "type.googleapis.com/google.commentanalyzer.v1alpha1.Error",\n        "errorType": "LANGUAGE_NOT_SUPPORTED_

 89%|██████████████████████████████████████████████████████▎      | 492/552 [6:36:00<38:57, 38.96s/it]ERROR:root:The following error occured: 
({'vary': 'Origin, X-Origin, Referer', 'content-type': 'application/json; charset=UTF-8', 'date': 'Fri, 28 Jan 2022 17:21:50 GMT', 'server': 'ESF', 'cache-control': 'private', 'x-xss-protection': '0', 'x-frame-options': 'SAMEORIGIN', 'x-content-type-options': 'nosniff', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"', 'transfer-encoding': 'chunked', 'status': '400', 'content-length': '484', '-content-encoding': 'gzip'}, b'{\n  "error": {\n    "code": 400,\n    "message": "Attribute TOXICITY does not support request languages: fil",\n    "status": "INVALID_ARGUMENT",\n    "details": [\n      {\n        "@type": "type.googleapis.com/google.commentanalyzer.v1alpha1.Error",\n        "errorType": "LANGUAGE_NOT_SUPPORTED_BY_A